# 实验报告


## 一、 代码设计逻辑
### 1. 整体描述
- 整体声明
    - 定义了一个共享缓冲区 buffer、添加和移除元素的位置 add 和 rem、缓冲区中元素数量 num、以及 done 标志位帮助线程A进行添加数字
    - 定义了三个信号量：mutex m、conditional variables c_cons 和 c_prod 并初始化，分别用于实现线程同步和互斥。
    - 定义了六个全局变量 countB、sumB、countC、sumC、countD 和 sumD，用于记录 Thread B、C、D 的统计结果。
- main函数中
    - 生成了4个线程。作为生产者向共享缓冲区中添加数字的线程A、作为消费者从共享缓冲区中移除数字并按条件进行统计的线程B、C、D。
        - Thread B 和 Thread C 有相似的功能，在 Thread B 中会对能被 pb 整除的数字进行筛选并进行统计，而 Thread C 则对能被 pc 整除的数字进行筛选并进行统计。
        - Thread D 对于不能被 pb 和 pc 整除的数字进行统计。
        - 只有向线程B和线程C传递了参数，方便利用同一个函数接口threadBC()实现相应功能  
    - 在所有线程结束后回收并输出最终统计结果
- threadA函数  
    - 通过使用 mutex 来保证线程同步和互斥
        - 如果缓冲区已满，调用 pthread_cond_wait() 函数等待信号 c_prod 释放 mutex 并进入睡眠状态，使其他线程能够向缓冲区中添加元素。
        - 如果缓冲区不满，则添加一个新元素到缓冲区，并更新 add 和 num，然后使用 pthread_cond_broadcast() 发信号通知所有等待 c_cons 的线程。
- threadBC函数
    - while循环内部，首先需要获取到参数p和q，
    - 然后使用 mutex 来保证线程同步和互斥，如果缓冲区是空的或下一个要移除的元素不符合条件，则生产者需要调用 pthread_cond_wait() 函数等待信号 c_cons 来释放 mutex 并睡眠。
    - 如果能被 p 整除，则将其从缓冲区中移除，并对统计变量 countB、sumB、countC 和 sumC 进行更新。最后，发送信号 c_prod 给等待该信号的生产者线程。
- threadD函数
    - 使用 mutex 来保证线程同步和互斥，
        - 如果缓冲区是空的或下一个要移除的元素不符合条件，则生产者需要调用 pthread_cond_wait() 函数等待信号 c_cons 来释放 mutex 并进入睡眠状态。
        - 如果不能被 pb 和 pc 整除，则将其从缓冲区中移除，并对统计变量 countD 和 sumD 进行更新。最后，发送信号 c_prod 给等待该信号的生产者线程。

### 2. 共享变量的多线程竞争与解决思路

通常在使用共享变量的多线程程序中，互斥锁(mutex)和条件变量(conditional variables)被用来进行线程同步和互斥。在本代码中，互斥锁被用作在临界区（critical section）中进行线程互斥，而条件变量则用来在线程之间传递信号并在生产者和消费者之间实现同步。例如：Thread A 通过发送 c_prod 信号来唤醒等待 c_cons 信号的 Thread B、C 和 D。而在 Thread B、C 和 D 中，则是通过发送 c_prod 信号来唤醒等待 c_cons 信号的 Thread A。这种同步机制是为了保证每个线程能够在正确的时刻执行，以避免出现数据竞争和不一致的问题。

## 二、测试示例
- ./main 10 5 3
    - Thread B, 2, 15
    - Thread C, 3, 18
    - Thread D, 5, 22
- ./main 100 5 3
    - Thread B, 20, 1050
    - Thread C, 33, 1683
    - Thread D, 53, 2632
- ./main 200 5 3
    - Thread B, 40, 4100
    - Thread C, 66, 6633
    - Thread D, 107, 10732
- ./main 200 10 5
    - Thread B, 20, 2100
    - Thread C, 40, 4100
    - Thread D, 160, 16000
- ./main 200 10 3
    - Thread B, 20, 2100
    - Thread C, 66, 6633
    - Thread D, 120, 11997